In [1]:
import os

In [2]:
list_file = list(os.walk('./make'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
list_file.sort()
    
len(list_file)

In [3]:
import pandas as pd

In [4]:
df = pd.DataFrame()

df

""


In [5]:
from tqdm import tqdm
from bs4 import BeautifulSoup
from lxml import etree
import json

In [6]:
for file in tqdm(list_file, desc='Progress', ncols=77):
    with open('./make/' + file, 'r', encoding='UTF-8') as f:
        html = f.read()
        
        # = = = = = = = = = = = = = = =
        
        soup = BeautifulSoup(html, 'lxml')
        
        # = = = = = = = = = = = = = = =
        
        html = etree.HTML(str(soup))
        
        list_make = [make.strip() for make in html.xpath('//div[@data-manufacturer-selector]/text()')]
        
        # = = = = = = = = = = = = = = =
        
        list_make_code = [code.strip() for code in html.xpath('//div[@data-manufacturer-selector]/@data-tab')]
        
        # = = = = = = = = = = = = = = =
        
        df_temp = pd.DataFrame({'Part_Code': file.replace('.txt', ''),
                                'Make': list_make,
                                'Make_Code': list_make_code,
                                'Model_Url': ['https://www.atp-autoteile.de/de/product/vehicles/' + file.replace('.txt', '') + '/' + code for code in list_make_code]})
        
        # = = = = = = = = = = = = = = =
        
        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

# = = = = = = = = = = = = = = =

df['Code'] = [part_code + ';' + model_code for part_code, model_code in zip(df['Part_Cdoe'].tolist(), df['Make_Code'].tolist())]
        
df = df.drop_duplicates(ignore_index=True)
df = df.sort_values(by=['Part_Code', 'Make'], ascending=[True, True]).reset_index(drop=True)
df.to_excel('./make.xlsx', index=False)

df

Progress: 100%|█████████████████████████| 1366/1366 [00:02<00:00, 596.15it/s]


,Part_Code,Make,Make_Code,Model_Url
0,1000392,VW,9992,https://www.atp-autoteile.de/de/product/vehicl...
1,1016832,Audi,9950,https://www.atp-autoteile.de/de/product/vehicl...
2,1016832,Seat,9983,https://www.atp-autoteile.de/de/product/vehicl...
3,1016832,Skoda,9984,https://www.atp-autoteile.de/de/product/vehicl...
4,1016832,VW,9992,https://www.atp-autoteile.de/de/product/vehicl...
...,...,...,...,...
2490,989599,Peugeot,9978,https://www.atp-autoteile.de/de/product/vehicl...
2491,989696,BMW,9953,https://www.atp-autoteile.de/de/product/vehicl...
2492,98999,Hyundai,10004,https://www.atp-autoteile.de/de/product/vehicl...
2493,98999,KIA,10005,https://www.atp-autoteile.de/de/product/vehicl...
